### Context Manager

**Syntax:**
```py
with ctx() as cx:
    pass
    # code to execute
```

**Exaple:**
```py
with open('data.csv') as f:
    for line in f.lines():
        print(line)
## f: closes automatically ##
```

**_Loose translation:_**
```py
cx = ctx()
cx.__enter__()
try:
    pass
    # code to execute
finally:
    cx.__exit__()
```

**_To create a context manager:_**
```py
class ctx:
    def __enter__(self):
        pass # initial operations like connect to internet
    
    def __exit__(self, *_, **__):
        pass # teardown operations like close the connection
             # this executes if __enter__ is success...
```

#### Example:

In [18]:
from sqlite3 import connect

In [19]:
class temptable:
    def __init__(self, cursor):
        self.cursor = cursor
        
    def __enter__(self):
        self.cursor.execute('create table temp_table (x int, y int);')
        for x in range(10):
            self.cursor.execute('insert into temp_table (x, y) values ({x}, {y});'.format(x=x, y=x * x))
    
    def __exit__(self, *_, **__):
        self.cursor.execute('drop table temp_table;')

In [20]:
with connect('datasets/xy.db') as conn:
    cursor = conn.cursor()
    with temptable(cursor): # using the ctx manager
        for row in cursor.execute('select * from temp_table;'):
            print(row)
    cursor.execute('select * from temp_table;') 
    # out of context temptable temp_table already dropped by __exit__

(0, 0)
(1, 1)
(2, 4)
(3, 9)
(4, 16)
(5, 25)
(6, 36)
(7, 49)
(8, 64)
(9, 81)


OperationalError: no such table: temp_table

🔝🔝 `temp_table` not found because 
```py
 cursor.execute('select * from temp_table;')
```
statement is out of `temptable` context `__exit__` function already executed and droped the table